In [3]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import MPNetTokenizer, MPNetModel
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import os
import numpy as np

model_path = '/data/gunsbrother/prjs/ltvu/everything/sbert_finetune/outputs/batch/2024-06-07/17-03-43/lit/103204/checkpoints/step=3834-nlq_R5@0.3=0.0000.ckpt'
model = SentenceTransformer('all-mpnet-base-v2').cuda().eval()
model.load_state_dict(
    {k.replace('model.model.', '0.auto_model.'): v for k, v in torch.load(model_path)['state_dict'].items()},
    strict=False
)

In [4]:
p_caps_dir = Path('/data/gunsbrother/prjs/ltvu/everything/sbert_finetune/data/captions/llava-v1.6-34b/global')
p_caps = list(p_caps_dir.glob('*.json'))

annotations = json.loads(Path(os.path.join('/data/soyeonhong/GroundVQA/data/unified/annotations.NLQ_val.json')).read_text())
valid_video_id = [annotation['video_id'] for annotation in annotations]

In [5]:
cos_sims_all = []
for p_cap in p_caps:
    if p_cap.stem in valid_video_id:

        cap_data = json.load(p_cap.open())['answers']
        # caps = [entry[2] for entry in cap_data]

        embeddings = model.encode(caps)

        cos_sim = embeddings @ embeddings.T
        
        adjacent_cos_sim = []
        for idx in range(1, len(embeddings) - 1):
            adjacent_cos_sim.append(cos_sim[idx, idx+1])

        cos_sims_all.append(np.mean(np.array(adjacent_cos_sim)))
        

In [6]:
np.mean(np.array(cos_sims_all))

0.71322966